In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

In [9]:
class Problem(object):
    """
    The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass.
    """

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [10]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.

In [11]:
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [12]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [13]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [29]:
class FarmerFoxChickenSeed(Problem):
  # initial state set to all 4 being on the left side of the river
  # goal state set to all 4 being on the right side of the river
  def __init__(self, initial=(0, 0, 0, 0), goal=(1, 1, 1, 1)):
    Problem.__init__(self, initial, goal)

  def is_valid(self, state):
    # checking each action to make sure the move is valid and game rules are followed
    farmer, fox, chicken, seed = state
    if farmer != fox and fox == chicken:
      return False
    if farmer != chicken and chicken == seed:
      return False
    return True

  def actions(self, state):
    # defining possible actions for the farmer to take
    farmer, fox, chicken, seed = state
    actions = []

    if self.is_valid((1 - farmer, fox, chicken, seed)):
      actions.append(('farmer',))

    if farmer == fox and self.is_valid((1 - farmer, 1-fox, chicken, seed)):
      actions.append(('farmer', 'fox'))

    if farmer == chicken and self.is_valid((1 - farmer, fox, 1-chicken, seed)):
      actions.append(('farmer', 'chicken'))

    if farmer == seed and self.is_valid((1 - farmer, fox, chicken, 1-seed)):
      actions.append(('farmer', 'seed'))

    return actions
  
  def result(self, state, action):
    # defining the result of the action taken by the farmer
    farmer, fox, chicken, seed = state
    new_state = list(state)

    for item in action:
      if item == 'farmer':
        new_state[0] = 1 if farmer == 0 else 0
      elif item == 'fox':
        new_state[1] = 1 if fox == 0 else 0
      elif item == 'chicken':
        new_state[2] = 1 if chicken == 0 else 0
      elif item == 'seed':
        new_state[3] = 1 if seed == 0 else 0

    return tuple(new_state)
  
  def is_goal(self, state):
    return state == self.goal 
  
  def h(self, node):
    # our h function differnece between all items being on the right and what is currently on the right
    return sum(node.state) - sum(self.goal)
  

### Trying different search trees
##### a* search is the optimal solution and finds the shortest path to the solution
##### it uses a heuristic function which shows how close we are to the goal

In [30]:
problem = FarmerFoxChickenSeed()
solution = astar_search(problem)
print(solution)
if solution != failure:
  print('path cost:', solution.path_cost)
  print('path:', path_actions(solution))
  print('path:', path_states(solution))
else:
  print('failure')

(0, 0, 0, 0)
(1, 0, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 0, 0)
(1, 0, 1, 1)
(0, 0, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
<(1, 1, 1, 1)>
path cost: 7
path: [('farmer', 'chicken'), ('farmer',), ('farmer', 'fox'), ('farmer', 'chicken'), ('farmer', 'seed'), ('farmer',), ('farmer', 'chicken')]
path: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 1, 1, 0), (0, 1, 0, 0), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]


In [31]:
problem = FarmerFoxChickenSeed()
solution = greedy_bfs(problem)
print(solution)
if solution != failure:
  print('path cost:', solution.path_cost)
  print('path:', path_actions(solution))
  print('path:', path_states(solution))
else:
  print('failure')

(0, 0, 0, 0)
(1, 0, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 0, 0)
(1, 0, 1, 1)
(0, 0, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
<(1, 1, 1, 1)>
path cost: 7
path: [('farmer', 'chicken'), ('farmer',), ('farmer', 'fox'), ('farmer', 'chicken'), ('farmer', 'seed'), ('farmer',), ('farmer', 'chicken')]
path: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 1, 1, 0), (0, 1, 0, 0), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]


In [32]:

problem = FarmerFoxChickenSeed()
solution =  breadth_first_bfs(problem)
print(solution)
if solution != failure:
  print('path cost:', solution.path_cost)
  print('path:', path_actions(solution))
  print('path:', path_states(solution))
else:
  print('failure')

(0, 0, 0, 0)
(1, 0, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(1, 0, 1, 1)
(0, 1, 0, 0)
(0, 0, 0, 1)
(1, 1, 0, 1)
(0, 1, 0, 1)
<(1, 1, 1, 1)>
path cost: 7
path: [('farmer', 'chicken'), ('farmer',), ('farmer', 'fox'), ('farmer', 'chicken'), ('farmer', 'seed'), ('farmer',), ('farmer', 'chicken')]
path: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 1, 1, 0), (0, 1, 0, 0), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]


In [33]:

problem = FarmerFoxChickenSeed()
solution =  depth_first_bfs(problem)
print(solution)
if solution != failure:
  print('path cost:', solution.path_cost)
  print('path:', path_actions(solution))
  print('path:', path_states(solution))
else:
  print('failure')

(0, 0, 0, 0)
(1, 0, 1, 0)
(0, 0, 1, 0)
(1, 1, 1, 0)
(0, 1, 0, 0)
(1, 1, 0, 1)
(0, 1, 0, 1)
<(1, 1, 1, 1)>
path cost: 7
path: [('farmer', 'chicken'), ('farmer',), ('farmer', 'fox'), ('farmer', 'chicken'), ('farmer', 'seed'), ('farmer',), ('farmer', 'chicken')]
path: [(0, 0, 0, 0), (1, 0, 1, 0), (0, 0, 1, 0), (1, 1, 1, 0), (0, 1, 0, 0), (1, 1, 0, 1), (0, 1, 0, 1), (1, 1, 1, 1)]
